In [19]:
import pandas as pd
import numpy as np
import altair as alt
import os, random
from Bio import SeqIO
import gensim 
from gensim.models import Word2Vec


## Defining paths for each and every omic

In [13]:
path_root_data = os.path.join ('..', 'Data', 'Extracted', 'Databases')
path_all_fasta = os.path.join (path_root_data, 'fasta_files', 'AllBins')
path_normalised_metabolomics = os.path.join (path_root_data, 'Metabolomics', 'Normalised_Tables')
path_model_save_root = 'Saved_models'


In [20]:
num_of_mags = len([i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')])
SEED = 42
END = 25
num_of_workers = 3
random.seed(SEED)
np.random.seed(SEED)

---
## Importing MAGs
---

In [26]:
# Function that splits each genome into k-mers thus creating even longer sentence (MAG)
# It returns tokenized genome i.e. [kmer, kmer,...]
def split_genome (genome, k = 5):
    new_genome = []
    n = len(genome)
    
    if n-k <=0:
        return genome
    else:
        for i in range(n-k):
            new_genome.append(genome[i:i+k])
        
        return new_genome


def vectorize_one_mag (one_mag, w2v_model):
    
    # We have to generate vectors for each word in one MAG and then create vector representation of that MAG
    # by averaging vectors of its words
    zero_vector = np.zeros(w2v_model.vector_size)
    word_vectors = []
    one_mag_vector = []
    
    for sentence in one_mag:
        for word in sentence:
            if word in w2v_model.wv:
                try:
                    word_vectors.append(w2v_model.wv[word])
                except KeyError:
                    print ('Key Error')
                    continue
    
    if word_vectors:
        word_vectors = np.asarray(word_vectors)
        one_mag_vector = word_vectors.mean (axis=0)
    
    else:
        one_mag_vector = zero_vector
    
    return one_mag_vector


# Function that vectorizes a MAG (document) with a pretrained word2vec model. It returns vector representation of a given MAG
# Vectorization is done by averaging word (k-mer) vectors for the whole document (MAG)
def vectorize_mags (w2v_model, end = 25):
    
    print ('Vectorizing MAGs')
    
    fasta_files = [i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')]
    list_of_mag_vectors = []
    
    # This was done so that I could work with first 'end' FASTA files only. Otherwise, I should just remove: i, and enumerate
    for i, fasta_file_name in enumerate(fasta_files):
        
        if i == end:
            break
        
        else:
            with open(os.path.join(path_all_fasta, fasta_file_name), 'r') as input_file:
                
                one_mag = []
                for fasta_string in SeqIO.parse(input_file, "fasta"):
                    
                    # Get kmers of a genome and create a sentence (list of words)
                    temp_kmers = split_genome (str(fasta_string.seq))
                    
                    # Create a document (list of sentences)
                    one_mag.append(temp_kmers)
                    
                # Vectorizing MAGs one by one
                list_of_mag_vectors.append (vectorize_one_mag (one_mag, w2v_model))
    
    print ('Finished vectorizing')
    
    return list_of_mag_vectors
    
    

# If one wants to import MAGs in order to vectorize them, one should use start argument in order to skip first 'start' MAGs
# If one wants to import MAGs to train word2vec model, one should use only end argument, so that first 'end' MAGs are used for training
# Todo: Implement randomisation in picking MAGs for training, and don't use first 'start' MAGs for training
# (create list of indexes from 0 to 1364, use sklearn split train test, then traverse directory and use only MAGs with indexes in train for training w2v)
def import_mags_and_build_model (end = 25, path_all_fasta = path_all_fasta):
    
    print ('Importing MAGs')
    
    # There are 1364 MAGs enclosed in FASTA files
    # I have to traverse every FASTA file, and in each file every sequence
    
    fasta_files = [i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')]
    
    # This was done so that I could work with first 100 FASTA files only. Otherwise, I should just remove: i, and enumerate
    for i, fasta_file_name in enumerate(fasta_files):
        
        if i == end:
            break
        
        else:
            with open(os.path.join(path_all_fasta, fasta_file_name), 'r') as input_file:
                
                one_mag = []
                for fasta_string in SeqIO.parse(input_file, "fasta"):
                    
                    # Get kmers of a genome and create a sentence (list of words)
                    temp_kmers = split_genome (str(fasta_string.seq))
                    
                    # Create a document (list of sentences)
                    one_mag.append(temp_kmers)
                    
                # If we do not have a model, we build one
                if i == 0:
                    print ('Building w2v model')
                    # We build our model on the first MAG
                    w2v_model = Word2Vec (sentences = one_mag, size = 100, workers = num_of_workers, seed=SEED)
                
                # Else we just expand its vocabulary
                else:
                    # Now we expand our vocabulary
                    w2v_model.build_vocab (one_mag, update = True)
                    
    print ('Finished building')
    
    return w2v_model


def train_model (w2v_model, epochs, end = 25):
    
    print ('Starting model training')
    
    # There are 1364 MAGs enclosed in FASTA files
    # I have to traverse every FASTA file, and in each file every sequence
    
    fasta_files = [i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')]
    
    # This was done so that I could work with first 100 FASTA files only. Otherwise, I should just remove: i, and enumerate
    for i, fasta_file_name in enumerate(fasta_files):
        
        if i == end:
            break
        
        else:
            with open(os.path.join(path_all_fasta, fasta_file_name), 'r') as input_file:
                
                one_mag = []
                for fasta_string in SeqIO.parse(input_file, "fasta"):
                    
                    # Get kmers of a genome and create a sentence (list of words)
                    temp_kmers = split_genome (str(fasta_string.seq))
                    
                    # Create a document (list of sentences)
                    one_mag.append(temp_kmers)
                    
                
                w2v_model.train (one_mag, total_examples = w2v_model.corpus_count, epochs = epochs)
                    
    print ('Model training finished')
    
    return w2v_model


---
## Importing Metabolomic data

---

In [4]:
metabolomics_file_name = os.path.join(path_normalised_metabolomics, os.listdir(path_normalised_metabolomics)[0])
metabolomics_df = pd.read_csv (metabolomics_file_name, delimiter = '\t')
metabolomics_df

,Metabolite,tp,date,type,known_type,means,medians,sds,N,se,ci,type2,measurement,KEGG.Compound.ID,Chebi.Name,Chebi.Name_combined
0,!Phosphoric_acid_3TMS,D1,2012-04-17,snp,known,0.757113,0.737943,0.098650,3,0.056956,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
1,!Phosphoric_acid_3TMS,D10,2012-01-25,snp,known,0.720935,0.873509,0.266593,3,0.153918,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
2,!Phosphoric_acid_3TMS,D11,2012-03-22,snp,known,0.625047,0.652268,0.057820,3,0.033382,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
3,!Phosphoric_acid_3TMS,D12,2012-01-19,snp,known,0.964956,0.941878,0.053754,3,0.031035,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
4,!Phosphoric_acid_3TMS,D13,2011-05-13,snp,known,0.963984,0.974479,0.071659,3,0.041373,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18865,Valine_2TMS,D51,2011-08-11,bp,known,0.704821,0.478251,0.428368,3,0.247319,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine
18866,Valine_2TMS,D6,2011-06-03,bp,known,1.085888,1.197471,0.487484,3,0.281449,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine
18867,Valine_2TMS,D7,2012-04-04,bp,known,2.958023,2.813145,0.341213,3,0.197000,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine
18868,Valine_2TMS,D8,2012-02-08,bp,known,2.515980,2.322248,0.618556,3,0.357123,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine


In [5]:
metabolomics_df['date'] = pd.to_datetime(metabolomics_df['date'])
metabolomics_df.insert (0, 'date', metabolomics_df.pop('date'))
metabolomics_df.sort_values ('date', inplace = True, ignore_index = True)
metabolomics_df

,date,Metabolite,tp,type,known_type,means,medians,sds,N,se,ci,type2,measurement,KEGG.Compound.ID,Chebi.Name,Chebi.Name_combined
0,2011-03-21,Leucine_2TMS,D37,sp,known,1.440122,1.489979,0.146440,3,0.084547,4.302653,extracellular,polar,"C16439,C00123,C01570",leucine;L-leucine;D-leucine,leucine;L-leucine;D-leucine
1,2011-03-21,No match: 1181.78_EM_SP_D1_1_223,D37,sp,unknown,3.778469,3.801417,1.924377,3,1.111040,4.302653,extracellular,polar,NaN,NaN,NaN
2,2011-03-21,unknown#emu_WW_2144.25,D37,bnp,unknown,0.949545,0.947209,0.065422,3,0.037771,4.302653,intracellular,nonpolar,NaN,NaN,NaN
3,2011-03-21,Unknown#bth_pae_001,D37,bp,unknown,2.095116,1.809891,1.916896,3,1.106721,4.302653,intracellular,polar,NaN,NaN,NaN
4,2011-03-21,No match: 2220.34_EM_BP_D1_1_192,D37,bp,unknown,1.631643,1.571125,0.168501,3,0.097284,4.302653,intracellular,polar,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18865,2012-05-03,Octadecanol_1TMS,D48,bnp,known,1.364105,1.457780,0.170311,3,0.098329,4.302653,intracellular,nonpolar,D01924,octadecan-1-ol,octadecan-1-ol
18866,2012-05-03,No match: 1447.93_EM_BP_D2_1_57,D48,bp,unknown,4.815774,1.160165,6.835474,3,3.946463,4.302653,intracellular,polar,NaN,NaN,NaN
18867,2012-05-03,Lactulose_8TMS,D48,bp,known,0.697489,0.684339,0.049632,3,0.028655,4.302653,intracellular,polar,C07064,lactulose,lactulose
18868,2012-05-03,No match: 2004.78_EM_BNP_D10_1_205,D48,bnp,unknown,0.785996,0.744529,0.139901,3,0.080772,4.302653,intracellular,nonpolar,NaN,NaN,NaN


## MAG examination

---
Important info:

> We can look at metagenome as a sentence which consists of words (that are represented with genomes). Therefore, we can create metagenomes as strings where words are seperated with space character (' '). This is one way of tackling this problem

To do:
1. Make kmers of some length, out of every word in every sentence
2. Encode every sentence with Word2Vec and CBOW (because it's faster than skip-gram)
3. Reduce dimensionality with PCA or with autoencoder
    > Maybe it is good to do the first 3 steps while importing, for each and every sentence. That way, I might use less memory
4. Cluster similar sentences (MAGs) to get rMAGs

> Important: this is not temporal data, since there are no timestamps!


In [6]:

# FOR CLUSTERING I SHOULD CREATE A DATAFRAME WITH MAGs INDEXES AND THEIR VECTOR REPRESENTATIONS

final_model = import_mags_and_build_model (end = END, path_all_fasta = path_all_fasta)


Importing MAGs
Building w2v model
Finished building


In [7]:
final_model.wv.most_similar('ACGGC')

[('CCGGC', 0.5800316333770752),
 ('TCGGC', 0.5560053586959839),
 ('ACGGA', 0.5357427597045898),
 ('GCGGC', 0.5319786667823792),
 ('ACGGG', 0.5190925598144531),
 ('ACGGT', 0.4692608118057251),
 ('CTACG', 0.30135977268218994),
 ('ACGAC', 0.2825249433517456),
 ('TACGA', 0.2640616297721863),
 ('AACGA', 0.2519393265247345)]

In [12]:
# Train model
epochs = 10

final_model = train_model (final_model, epochs = epochs, end = END)


Starting model training
Model training finished


In [14]:
final_model.wv.save_word2vec_format(os.path.join (path_model_save_root, 'model_25.bin'), binary=True) 

Now I should vectorize documents with this model. For further use, I could save this model's weights, and use it to vectorize all mags. That would take a lot, but every MAG will have its vector representation
> This could be done by importing one MAG at a time, then tokenizing it (like before), then getting vector representations of that MAG's sentences (genomes) and then finding the vector representation of the whole MAG (document). If I do that for one MAG at a time, There is no need to worry about memory


In [25]:
list_of_mag_vectors = vectorize_mags (final_model, end = END)

Vectorizing MAGs


In [32]:
list_of_mag_vectors[0:2]

[array([ 1.2695041 , -1.3016177 ,  0.65977436,  0.80642366,  1.7741423 ,
        -0.34343222, -0.7791513 , -0.21512581, -0.18608665, -1.4746015 ,
        -0.83035123, -0.11485589,  0.8619527 ,  0.07787611, -1.0242358 ,
        -0.85770607, -1.024094  ,  0.46239102,  2.9101133 ,  0.3759784 ,
        -0.42531765,  1.3783201 , -0.08274465, -0.16104594, -1.5054408 ,
        -1.2894931 , -0.99346983,  0.01654907,  0.80869704, -1.9024653 ,
         1.0351242 ,  0.71593   ,  1.1183119 , -0.6273121 , -1.2179893 ,
         0.6693737 ,  1.6966379 ,  0.66082835,  0.5160004 ,  0.3739503 ,
         1.4906927 ,  1.2522779 ,  0.9794797 , -0.5130619 , -0.5718369 ,
         1.9109504 ,  1.1832289 , -1.4082754 , -1.2452774 , -1.1157957 ,
        -0.54767996,  1.4872566 , -0.76184076, -1.5441176 , -2.2053912 ,
        -0.8439441 ,  0.82276267,  1.8204472 ,  0.45814872,  0.12408362,
        -1.2742198 , -0.01376653,  0.1029577 , -2.7019987 ,  1.4996148 ,
        -0.6873514 , -0.39449534,  2.92373   ,  0.7

In [35]:
mags_df = pd.DataFrame (list_of_mag_vectors)
mags_df


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.269504,-1.301618,0.659774,0.806424,1.774142,-0.343432,-0.779151,-0.215126,-0.186087,-1.474602,...,-1.693101,-1.635854,-1.960710,0.339103,-0.025599,0.260683,0.163350,1.253483,0.558545,-1.454138
1,-1.219352,0.858967,-2.209150,-0.584687,-1.285293,0.307079,-0.010814,1.042626,-0.409562,1.655249,...,1.111762,1.085674,1.186331,-1.579450,-0.355581,1.330155,0.142522,0.414558,2.009863,0.834968
2,1.455337,-0.348460,2.492809,-0.002663,1.328323,-0.874232,-0.157595,0.838077,0.904898,-1.876874,...,-3.277988,-1.223678,-2.357131,0.819557,-0.908406,0.079156,1.186641,0.651305,-0.033716,-1.122361
3,-1.128447,0.772527,-2.491378,-0.651029,-1.623185,0.663864,0.125967,1.104853,-0.519641,2.120744,...,1.073344,1.364996,1.529100,-1.807320,-0.537589,1.110650,-0.042950,0.446592,1.589758,0.847480
4,-1.036939,0.809364,-2.464699,-0.687017,-1.576593,0.638427,0.118440,1.119194,-0.525833,2.067393,...,1.060543,1.332533,1.429077,-1.721097,-0.570125,1.051956,-0.089572,0.433013,1.584785,0.764304
5,-0.547059,0.894858,-1.755465,-0.490597,-1.380145,0.359310,-0.060178,0.771358,-0.472385,1.670432,...,0.283317,1.162495,1.423480,-1.511162,-0.741800,0.205500,0.125913,0.088043,0.769423,0.282571
6,1.733710,-0.566986,2.949093,0.390996,2.418676,-1.398353,-0.696868,0.764553,1.415131,-3.074588,...,-3.647666,-2.091958,-3.698048,1.739185,-0.828084,0.024867,1.245571,0.942139,-0.019091,-1.623251
7,1.902613,-1.295641,2.932941,0.292558,2.542777,-0.784014,-0.564508,0.691168,0.872062,-2.692172,...,-3.710976,-2.149990,-3.701959,1.104722,-0.588351,0.193837,1.070350,1.491803,-0.170504,-1.562872
8,0.986385,-0.628855,0.235144,0.294113,0.755645,-0.287721,-0.565301,-0.137433,-0.318865,-0.364859,...,-1.473106,-0.788323,-0.883935,-0.450523,-0.389556,-0.204728,0.018448,0.864524,0.220383,-1.094242
9,0.685016,-0.085557,-0.214265,0.292677,0.134052,-0.131870,-0.285285,0.141165,-0.274307,-0.183934,...,-0.833542,-0.349754,-0.144961,-0.388498,-0.842064,-0.071577,0.339474,0.374968,0.558935,-0.894052


---
## Clustering
---